In [ ]:
import keras
keras.__version__

# Wizualizacja efektów uczenia konwolucyjnych sieci neuronowych

Często słyszy się, że modele uczenia głębokiego są jak „czarne skrzynki”: uczą się reprezentacji, których ekstrakcja jest trudna do wykonania i trudno jest je przedstawić w formie czytelnej z punktu widzenia człowieka. Stwierdzenie to dla niektórych modeli uczenia głębokiego można uznać za częściowo prawdziwe, ale z pewnością nie jest ono prawdziwe w przypadku pracy z konwolucyjnymi sieciami neuronowymi. Reprezentacje uczone przez konwolucyjne sieci neuronowe można z łatwością przedstawić w sposób graficzny. Wynika to w dużej mierze z tego, że są to reprezentacje koncepcji graficznych. Od 2013 r. rozwinięto wiele technik wizualizacji i interpretowania takich reprezentacji. Nie będę opisywał ich wszystkich, ale przedstawię Ci trzy najprostsze rozwiązania tego typu:

* Wizualizacja pośrednich danych wyjściowych (pośrednich aktywacji) — technika przydatna do zrozumienia tego, jak kolejne warstwy sieci konwolucyjnej przekształcają dane wejściowe i tego, co robią poszczególne filtry sieci.
* Wizualizacja filtrów sieci konwolucyjnej — technika umożliwiająca dokładne zrozumienie graficznego wzorca lub graficznej koncepcji, na którą reaguje każdy z filtrów sieci konwolucyjnej.
* Wizualizacja map ciepła aktywacji klas obrazu — technika przydatna do określenia części obrazu zidentyfikowanych jako należące do danej klasy — pozwala zlokalizować obiekty widoczne na obrazach.

Działanie pierwszej techniki — wizualizacji aktywacji — sprawdzimy na przykładzie małej sieci konwolucyjnej wytrenowanej od podstaw podczas podejmowania próby rozwiązania klasyfikacji obrazów psów i kotów w podrozdziale 5.2. Działanie dwóch kolejnych technik sprawdzimy na modelu VGG16 z podrozdziału 5.3.

## Wizualizacja pośrednich aktywacji

Wizualizacja pośrednich aktywacji polega na wyświetlaniu map cech, które są generowane przez różne warstwy konwolucyjne i łączące sieci na podstawie określonych danych wejściowych (dane wyjściowe warstwy są często określane mianem aktywacji, wynikiem zwracanym przez funkcję aktywacji). Umożliwia ona podgląd tego, jak dane wejściowe są rozkładane przez różne filtry wyuczone przez sieć. Chcemy dokonać wizualizacji map cech w przestrzeni trzech wymiarów: wysokości, szerokości i głębi (kanały). Każdy kanał koduje względnie niezależne cechy, a więc w celu wykonania poprawnej wizualizacji map tych cech należy przedstawić zawartość każdego kanału w formie niezależnego dwuwymiarowego obrazu. Zacznijmy od załadowania modelu zapisanego podczas rozwiązywania problemu z podrozdziału 5.2:

In [ ]:
from keras.models import load_model

model = load_model('cats_and_dogs_small_2.h5')
model.summary()  # W celu przypomnienia.

Teraz uzyskamy obraz wejściowy — zdjęcie kota niewchodzące w skład treningowego zbioru danych.

In [ ]:
#img_path = '/Users/fchollet/Downloads/cats_and_dogs_small/test/cats/cat.1700.jpg'
# Ustawka Technologiczna, 14.06.2019 -> przetestujcie również na zdjęciu cat.1700.jpg


# Wstępnie przetwarzamy obraz tak, aby uzyskał formę czterowymiarowego tensora.
from keras.preprocessing import image
import numpy as np

img = image.load_img(img_path, target_size=(150, 150))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
# Pamiętaj o tym, że model był trenowany 
# na danych przetworzonych właśnie w taki sposób.
img_tensor /= 255.

# Kształt tensora: (1, 150, 150, 3).
print(img_tensor.shape)

Czas wyświetlić zdjęcie:

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img_tensor[0])
plt.show()

W celu wyciągnięcia map cech, którym chcemy się przyjrzeć utworzymy model Keras, który będzie przyjmował na wejściu wsady składające się z obrazów i generował na swoim wyjściu aktywacje wszystkich warstw konwolucyjnych i warstw łączących. W tym celu skorzystamy z klasy Model pakietu Keras. Instancję modelu utworzymy korzystając z dwóch argumentów: tensora wejściowego (lub listy tensorów wejściowych) i tensora wyjściowego (lub listy tensorów wyjściowych). Utworzona w ten sposób klasa jest modelem Keras, który podobnie do opisanych wcześniej modeli sekwencyjnych (Sequential) mapuje określone dane wejściowe do określonych danych wyjściowych. Klasę Model od klasy Sequential odróżnia to, że umożliwia ona tworzenie modeli z wieloma wyjściami. Więcej informacji na temat klasy Model znajdziesz w podrozdziale 7.1.

In [ ]:
from keras import models

# Wyciąga dane wyjściowe ośmiu górnych warstw.
layer_outputs = [layer.output for layer in model.layers[:8]]
# Tworzy model zwracający dane wyjściowe generowane w wyniku przekazania określonych danych wejściowych.
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)

Model ten po skierowaniu do jego wejścia obrazu zwraca wartości aktywacji warstwy oryginalnego modelu. To pierwszy przykład modelu zwracającego wiele danych wyjściowych, który został zaprezentowany w tej książce. Dotychczas przedstawiałem modele, które przyjmowały dokładnie jeden obiekt wejściowy i zwracały jeden obiekt wyjściowy, ale ogólnie rzecz biorąc, model może przyjmować dowolną liczbę obiektów wejściowych i generować dowolną liczbę obiektów wyjściowych. Opisywany aktualnie model przyjmuje jeden obiekt wejściowy i generuje osiem obiektów wyjściowych (po jednym dla każdej warstwy aktywacji).

In [ ]:
# Zwraca listę pięciu tablic Numpy: po jednej tablicy dla każdej warstwy aktywacji.
activations = activation_model.predict(img_tensor)

Oto przykład aktywacji pierwszej warstwy konwolucyjnej wygenerowanej na podstawie wybranego przez nas zdjęcia kota:

In [ ]:
first_layer_activation = activations[0]
print(first_layer_activation.shape)

Jest to mapa cech o rozmiarach 148x148 składająca się z 32 kanałów. Przedstawmy w formie graficznej czwarty kanał aktywacji pierwszej warstwy oryginalnego modelu:

In [ ]:
import matplotlib.pyplot as plt

plt.matshow(first_layer_activation[0, :, :, 3], cmap='viridis')
plt.show()

Kanał ten wydaje się kodować informacje o wykryciu linii ukośnych. Spróbujmy przyjrzeć się siódmemu kanałowi — pamiętaj o tym, że we własnym kodzie informacje, które zawiera mój siódmy kanał, mogą znajdować się w innym kanale, ponieważ filtry tworzone przez warstwy konwolucyjne nie mają charakteru deterministycznego.

In [ ]:
plt.matshow(first_layer_activation[0, :, :, 30], cmap='viridis')
plt.show()

Warstwa ta wydaje się wykrywać „jasne zielone kropki” — jest to dobry sposób na zakodowanie wizerunku kocich oczu. Wygenerujmy kompletną wizualizację wszystkich warstw aktywacji naszej sieci. Wyciągniemy dane każdego kanału ośmiu map aktywacji i przedstawimy je na wykresie majacym formę jednego dużego zbioru obrazów (kanały zostaną umieszczone obok siebie).

In [ ]:
import keras

# Na wykresie umieścimy również nazwy warstw.
layer_names = []
for layer in model.layers[:8]:
    layer_names.append(layer.name)

images_per_row = 16

# Wyświetla mapy cech.
for layer_name, layer_activation in zip(layer_names, activations):
    # Liczba cech w mapie.
    n_features = layer_activation.shape[-1]

    # Mapa cech ma kształt (1, size, size, n_features).
    size = layer_activation.shape[1]

    # Tworzy kafelki kanałów aktywacji w tej macierzy.
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    # Tworzy poziomą siatkę składajacą się z filtrów.
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            # Przetwarzanie cechy w celu wygenerowania czytelnej wizualizacji.
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image

    # Wyświetla siatkę.
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    
plt.show()

Warto zauważyć tu kilka rzeczy:

* Pierwsza warstwa wykrywa różne krawędzie. Na tym etapie aktywacje zachowują prawie wszystkie informacje obecne w początkowym obrazie.
* Kolejne warstwy aktywacji stają się coraz bardziej abstrakcyjne i trudniej je zinterpretować graficznie. Mogą one zaczynać kodować wysokopoziomowe koncepcje takie jak „kocie ucho” i „kocie oko”. Wyższe reprezentacje danych zawierają coraz mnie informacji dotyczących wizualnej zawartości obrazu i jednocześnie coraz więcej informacji związanych z klasą obrazu.
* Wraz z głębią warstw maleje ich gęstość: wszystkie filtry pierwszej warstwy są aktywowane obrazem wejściowym, ale w kolejnych warstwach coraz więcej filtrów jest pustych. Oznacza to w obrazie brak wzorca zakodowanego przez dany filtr.


Właśnie odkryliśmy ważną cechę reprezentacji uczonych przez głębokie sieci neuronowe: wraz ze wzrostem głębokości warstwy rozpoznawane przez nią cechy stają się coraz bardziej abstrakcyjne. Aktywacje wyższych warstw zawierają coraz mniej informacji o obserwacji wejściowej, jednocześnie zawierają coraz więcej informacji o celu (klasie, do której należy obraz). Głęboka sieć neuronowa pełni funkcję potoku oczyszczającego informację — na wejściu tego potoku podawany jest obraz (w tym przypadku jest to obraz w formacie RGB), który w wyniku powtarzania przekształceń jest odzierany ze zbędnych informacji (np. wizualnych parametrów danego zdjęcia), co prowadzi do wzmocnienia przydatnych informacji, na podstawie których można ustalić klasę, do której należy obraz.

W taki sam sposób świat postrzegają ludzie i zwierzęta: po obserwowaniu jakiejś sceny przez kilka sekund człowiek zapamiętuje abstrakcyjne obiekty, które były w niej obecne (takie jak rower lub drzewo), ale nie pamięta konkretnego wyglądu tych obiektów. Spróbuj narysować rower z pamięci. Najprawdopodobniej będzie to problematyczne pomimo tego, że z pewnością zdarzyło Ci się w życiu widzieć rowery wielokrotnie. Spróbuj to zrobić teraz. Nasz mózg naprawdę działa, ucząc się abstrakcji bodźców wizualnych i odzierając je ze zbędnych szczegółów. Przez to zjawisko dość trudno jest zapamiętać detale wyglądu otaczających nas rzeczy.